# Projekt Machine Learning – Online Retail II
## Kompletny szkielet notebooka
---
## 0. Overview
**TODO:** Opis projektu, cel, dane, metody, plan pracy.


## 1. Wczytanie danych
**TODO:** Importy, ładowanie, wstępny opis.

## 2. EDA & Czyszczenie
**TODO:** Braki, korelacje, błędy, outliery, czyszczenie.

## 3. Feature Engineering (RFM)
**TODO:** R, F, M, agregacje, snapshot_date.

## 4. Outliers – Isolation Forest
**TODO:** Wykrywanie klientów odstających.

## 5. Standaryzacja
**TODO:** StandardScaler na cechach liczbowych.

## 6. Feature Selection
**TODO:** LASSO, Permutation Importance, PCA.

## 7. Clustering – KMeans
**TODO:** Elbow, Silhouette, klastry i ich opis.

## 8. Modelowanie supervised
**TODO:** Target high-value, modele baseline i docelowe.

## 9. Optuna – tuning modeli
**TODO:** Strojenie hiperparametrów XGBoost i LogReg.

## 10. Interpretacja – SHAP, PDP
**TODO:** SHAP values, partial dependence, wnioski.

## 11. Podsumowanie
**TODO:** Wnioski biznesowe i techniczne.

In [ ]:
# TODO: Kod wykonujący dany krok
pass